<a href="https://colab.research.google.com/github/krxa14/real-time-chess-vision/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Download the 's4lman/chess-pieces-dataset-85x85' Kaggle dataset.

## Download Kaggle Dataset

### Subtask:
Download the 's4lman/chess-pieces-dataset-85x85' Kaggle dataset using `kagglehub.dataset_download` to get the image files for chess piece recognition.


**Reasoning**:
To download the Kaggle dataset, I need to import the `kagglehub` library and then use its `dataset_download` function as specified in the instructions. This will allow me to access the chess piece images for further processing.



In [46]:
import kagglehub

dataset_path = kagglehub.dataset_download('s4lman/chess-pieces-dataset-85x85')
print(f"Dataset downloaded to: {dataset_path}")

100%|██████████| 5.95M/5.95M [00:00<00:00, 179MB/s]

Extracting files...
Dataset downloaded to: /root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2


**Reasoning**:
The previous step successfully downloaded the dataset, as indicated by the output. To continue with the overall goal of 'getting the image files for chess piece recognition', the next logical step is to inspect the contents of the downloaded dataset directory to understand its structure and locate the relevant image files.



In [47]:
from pathlib import Path
import os

# List the top-level contents of the downloaded dataset directory
download_dir = Path(dataset_path)
print(f"Contents of the downloaded dataset at '{download_dir}':")
for item in download_dir.iterdir():
    print(f"- {item.name}{'/' if item.is_dir() else ''}")

# Optionally, list contents of the first subdirectory if it exists and is a directory
first_subdir = next((item for item in download_dir.iterdir() if item.is_dir()), None)
if first_subdir:
    print(f"\nContents of the first subdirectory '{first_subdir.name}/':")
    for item in first_subdir.iterdir():
        print(f"  - {item.name}{'/' if item.is_dir() else ''}")

Contents of the downloaded dataset at '/root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2':
- data/

Contents of the first subdirectory 'data/':
  - knight/
  - bishop/
  - king/
  - pawn/
  - queen/
  - rook/


**Reasoning**:
The previous steps successfully downloaded the dataset and identified its structure, showing that image files are organized into subdirectories by piece type within the 'data/' folder. To 'get the image files' as per the subtask, the next logical step is to collect all image file paths from these subdirectories and store them in a structured way (e.g., a dictionary mapping piece names to lists of image paths).



In [48]:
from pathlib import Path
import os

image_files = {}
download_dir = Path(dataset_path) / 'data'

# Iterate through each subdirectory (chess piece type)
for piece_dir in download_dir.iterdir():
    if piece_dir.is_dir():
        piece_name = piece_dir.name
        image_files[piece_name] = []
        # Collect all image files in the current piece's directory
        for img_file in piece_dir.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                image_files[piece_name].append(str(img_file))

print(f"Collected {sum(len(v) for v in image_files.values())} image files categorized by piece type.")
print(f"Piece types found: {list(image_files.keys())}")
# Display the first few paths for one piece type as an example
if 'king' in image_files and image_files['king']:
    print(f"\nFirst 3 king images: {image_files['king'][:3]}")

Collected 517 image files categorized by piece type.
Piece types found: ['knight', 'bishop', 'king', 'pawn', 'queen', 'rook']

First 3 king images: ['/root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2/data/king/19.png', '/root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2/data/king/12.png', '/root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2/data/king/30.png']


## Unzip and Organize Dataset

### Subtask:
Organize the downloaded chess piece images by both piece type and color.


**Reasoning**:
I need to define a function that classifies the color of a chess piece from its image and then use this function to organize the images by both piece type and color. This function will involve loading images and analyzing their pixel data, for which `PIL` (Pillow) is a suitable library. I will also initialize a new dictionary to store the categorized images.



In [49]:
from PIL import Image
import numpy as np

def classify_piece_color_and_type(image_path, piece_type):
    """
    Classifies the color of a chess piece (white or black) from its image
    and returns a combined label string.
    """
    try:
        img = Image.open(image_path).convert('L') # Convert to grayscale
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return None
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None

    # Analyze a central region to avoid background noise/edges
    width, height = img.size
    # Consider a central square region, e.g., 50% of the image size
    crop_border_w = width // 4
    crop_border_h = height // 4
    cropped_img = img.crop((crop_border_w, crop_border_h, width - crop_border_w, height - crop_border_h))

    avg_brightness = np.array(cropped_img).mean()

    # A simple threshold to classify white vs black pieces. This might need tuning.
    # Assuming light pieces have higher average brightness than dark pieces.
    # The dataset images seem to have fairly consistent backgrounds, making this feasible.
    color_threshold = 128 # Mid-point of 0-255 grayscale range

    if avg_brightness > color_threshold:
        color = 'white'
    else:
        color = 'black'

    return f"{color}_{piece_type}"

# Initialize a new dictionary to store images categorized by piece type and color
categorized_images = {}

print("Function 'classify_piece_color_and_type' defined and 'categorized_images' dictionary initialized.")


Function 'classify_piece_color_and_type' defined and 'categorized_images' dictionary initialized.


**Reasoning**:
Now that the classification function is defined, I will iterate through the previously collected `image_files` dictionary, apply the `classify_piece_color_and_type` function to each image to determine its color and piece type, and then populate the `categorized_images` dictionary with these refined classifications. Finally, I will print a summary to verify the organization.



In [50]:
for piece_type, paths in image_files.items():
    for img_path in paths:
        categorized_label = classify_piece_color_and_type(img_path, piece_type)
        if categorized_label:
            if categorized_label not in categorized_images:
                categorized_images[categorized_label] = []
            categorized_images[categorized_label].append(img_path)

print("Summary of categorized images by piece type and color:")
for label, paths in categorized_images.items():
    print(f"- {label}: {len(paths)} images")

Summary of categorized images by piece type and color:
- white_knight: 35 images
- black_knight: 36 images
- white_bishop: 32 images
- black_bishop: 35 images
- white_king: 16 images
- black_king: 18 images
- white_pawn: 136 images
- black_pawn: 103 images
- black_queen: 17 images
- white_queen: 17 images
- white_rook: 35 images
- black_rook: 37 images


## Create Data Loaders and Augmentation

### Subtask:
Develop data loaders for the organized dataset and implement data augmentation techniques (e.g., rotations, flips, brightness adjustments) to increase the diversity of the training data and improve model robustness.


**Reasoning**:
The first instruction is to prepare the data for dataset creation. This involves extracting all image paths and their corresponding string labels from the `categorized_images` dictionary, then creating a mapping from these string labels to unique integer indices, and finally converting the string labels into numerical indices.



In [51]:
import torch

# 1. Prepare Data for Dataset Creation
all_image_paths = []
all_labels_str = []

for label_str, paths in categorized_images.items():
    all_image_paths.extend(paths)
    all_labels_str.extend([label_str] * len(paths))

# Create a dictionary to map string labels to integer indices
unique_labels = sorted(list(set(all_labels_str)))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# Convert string labels to numerical indices
all_labels_idx = [label_to_idx[label] for label in all_labels_str]

print(f"Total images: {len(all_image_paths)}")
print(f"Total labels (string): {len(all_labels_str)}")
print(f"Total labels (numerical): {len(all_labels_idx)}")
print(f"Unique labels mapping: {label_to_idx}")

# Convert all_labels_idx to a tensor
all_labels_idx = torch.tensor(all_labels_idx, dtype=torch.long)

Total images: 517
Total labels (string): 517
Total labels (numerical): 517
Unique labels mapping: {'black_bishop': 0, 'black_king': 1, 'black_knight': 2, 'black_pawn': 3, 'black_queen': 4, 'black_rook': 5, 'white_bishop': 6, 'white_king': 7, 'white_knight': 8, 'white_pawn': 9, 'white_queen': 10, 'white_rook': 11}


**Reasoning**:
Now that the data paths and numerical labels are prepared, the next step is to define the image transformation pipelines for both training and validation sets. This involves importing `torchvision.transforms` and `torch`, specifying mean and standard deviation for normalization, and composing various augmentation techniques for training data while keeping validation transformations minimal.



In [52]:
import torchvision.transforms as transforms
import torch

# 2. Define Image Transformations
# Define mean and std for normalization. Using ImageNet defaults or calculating
# from a sample of your dataset are common approaches. For simplicity, we'll use 0.5 for all channels.
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# Training transformations with data augmentation
train_transforms = transforms.Compose([
    transforms.Resize((85, 85)), # Resize all images to a consistent size
    transforms.RandomRotation(15), # Rotate by up to 15 degrees
    transforms.RandomHorizontalFlip(), # Randomly flip images horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Adjust brightness, contrast, saturation, hue
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize(mean=mean, std=std) # Normalize pixel values
])

# Validation transformations (typically only resizing and normalization)
val_transforms = transforms.Compose([
    transforms.Resize((85, 85)), # Resize to the same size as training
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize(mean=mean, std=std) # Normalize pixel values
])

print("Image transformation pipelines (train_transforms and val_transforms) defined.")

Image transformation pipelines (train_transforms and val_transforms) defined.


**Reasoning**:
Now that the image paths, numerical labels, and transformation pipelines are defined, the next step is to create a custom PyTorch Dataset class. This class will encapsulate how individual images are loaded, transformed, and paired with their corresponding labels, which is a fundamental component for training deep learning models in PyTorch.



In [53]:
from torch.utils.data import Dataset
from PIL import Image
import torch

# 3. Create a Custom PyTorch Dataset Class
class ChessPiecesDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

print("Custom PyTorch Dataset class 'ChessPiecesDataset' defined.")


Custom PyTorch Dataset class 'ChessPiecesDataset' defined.


**Reasoning**:
Now that the custom dataset class is defined, the next logical step is to split the dataset into training and validation sets, and then create PyTorch `DataLoader` instances for each. This is crucial for iterating over batches of data during model training and evaluation, applying the previously defined transformations, and managing batching, shuffling, and multiprocessing.



In [54]:
from torch.utils.data import random_split, DataLoader

# 4. Split Data and Instantiate Data Loaders

# Create a full dataset using the custom class without transformations initially
# Transforms will be applied to the subsets later.
full_dataset = ChessPiecesDataset(image_paths=all_image_paths, labels=all_labels_idx, transform=None)

# Define split ratios (e.g., 80% for training, 20% for validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Use random_split to divide the dataset into training and validation subsets
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

# Create new Dataset instances for train and validation subsets,
# applying the appropriate transforms to each.
# Note: When using random_split, the subsets are `torch.utils.data.Subset` objects.
# We need to manually apply the transforms to their `dataset` attribute
# or create new `ChessPiecesDataset` instances using their indices.
# A more direct way with `Subset` is to update the transform directly, but it's not ideal
# as it modifies the underlying full_dataset's transform if done incorrectly.
# The better approach for custom Dataset is to pass the subsets' indices
# and corresponding paths/labels to new ChessPiecesDataset objects.

# This method correctly assigns transforms to the new dataset objects created from subsets
# For `Subset` objects, their `indices` and `dataset` attributes are key.

# Extract paths and labels for the training subset
train_image_paths = [full_dataset.image_paths[i] for i in train_subset.indices]
train_labels = [full_dataset.labels[i] for i in train_subset.indices]

# Extract paths and labels for the validation subset
val_image_paths = [full_dataset.image_paths[i] for i in val_subset.indices]
val_labels = [full_dataset.labels[i] for i in val_subset.indices]

# Create new ChessPiecesDataset instances with specific transforms
train_dataset = ChessPiecesDataset(image_paths=train_image_paths, labels=train_labels, transform=train_transforms)
val_dataset = ChessPiecesDataset(image_paths=val_image_paths, labels=val_labels, transform=val_transforms)

# Define batch size and number of workers
batch_size = 32
num_workers = 2 # Often set to num_cores or 0 for debugging

# Instantiate DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Train DataLoader created with {len(train_loader)} batches of size {batch_size}.")
print(f"Validation DataLoader created with {len(val_loader)} batches of size {batch_size}.")


Training dataset size: 413
Validation dataset size: 104
Train DataLoader created with 13 batches of size 32.
Validation DataLoader created with 4 batches of size 32.


## Final Task

### Subtask:
Confirm that the Kaggle dataset has been successfully loaded, processed, and is ready for training a piece recognition model.


## Summary:

### Data Analysis Key Findings
*   The 's4lman/chess-pieces-dataset-85x85' Kaggle dataset was successfully downloaded and extracted to `/root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2`. It contains a total of 517 image files (JPG, JPEG, PNG) organized by piece type within a `data/` subdirectory.
*   Images were successfully categorized by both piece type and color using a custom function that analyzed the average brightness of a central image region. This resulted in 12 distinct categories (e.g., 'white\_knight', 'black\_pawn'). For instance, there are 35 'white\_knight' images, 36 'black\_knight' images, 136 'white\_pawn' images, and 103 'black\_pawn' images.
*   String labels for the 12 unique chess piece categories were mapped to numerical indices (0-11) for machine learning compatibility.
*   Image transformation pipelines were defined: `train_transforms` includes resizing, random rotation (15 degrees), random horizontal flip, color jitter, conversion to tensor, and normalization; `val_transforms` includes resizing, conversion to tensor, and normalization.
*   A custom `ChessPiecesDataset` class was implemented to efficiently load and preprocess images.
*   The dataset was split into a training set (413 samples, 80%) and a validation set (104 samples, 20%).
*   `DataLoader` objects were successfully created for both training (13 batches, shuffled) and validation (4 batches, not shuffled) sets, each with a batch size of 32.

### Insights or Next Steps
*   The dataset is fully prepared and augmented, making it ready for direct use in training a chess piece recognition model.
*   The implemented data augmentation techniques will enhance the model's ability to generalize by exposing it to diverse variations of the chess piece images during training.


# Task
## Subtask: Define CNN Architecture

### Instructions:
Define the architecture of a Convolutional Neural Network (CNN) suitable for image classification in PyTorch. The CNN should inherit from `torch.nn.Module` and include:
1.  **Convolutional Layers**: At least two convolutional layers with appropriate kernel sizes, padding, and activation functions (e.g., ReLU).
2.  **Pooling Layers**: Max-pooling layers after convolutional layers to reduce spatial dimensions.
3.  **Fully Connected Layers**: At least two fully connected (linear) layers for classification, with an activation function (e.g., ReLU) between them.
4.  **Output Layer**: A final fully connected layer that outputs the number of classes (which is 12, derived from `len(unique_labels)`).

### Reasoning:
To define the CNN architecture, I will use `torch.nn` to create the sequential layers that make up the network. This includes `nn.Conv2d` for feature extraction, `nn.MaxPool2d` for dimensionality reduction, and `nn.Linear` for the classification head. The `ReLU` activation function will be used for non-linearity. The input image size is 85x85, and there are 12 output classes. The architecture will be designed to handle these specifications.
```python
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Define CNN Architecture
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        # First convolutional layer
        # Input: 3 channels (RGB), Output: 32 channels, Kernel: 3x3, Padding: 1
        # Image size: 85x85x3 -> 85x85x32 (after conv and padding)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Max pooling layer
        # Image size: 85x85x32 -> 42x42x32 (after maxpool with kernel 2, stride 2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolutional layer
        # Input: 32 channels, Output: 64 channels, Kernel: 3x3, Padding: 1
        # Image size: 42x42x32 -> 42x42x64 (after conv and padding)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Max pooling layer
        # Image size: 42x42x64 -> 21x21x64 (after maxpool with kernel 2, stride 2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Third convolutional layer (optional, but good for deeper features)
        # Input: 64 channels, Output: 128 channels, Kernel: 3x3, Padding: 1
        # Image size: 21x21x64 -> 21x21x128 (after conv and padding)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        # Max pooling layer
        # Image size: 21x21x128 -> 10x10x128 (after maxpool with kernel 2, stride 2, integer division)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)


        # Fully connected layers
        # The input features to the first fully connected layer need to be calculated.
        # After conv3 and pool3, the spatial dimensions are approximately 10x10 and channels are 128.
        # So, 128 * 10 * 10 = 12800 features
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, num_classes) # Output layer with num_classes

    def forward(self, x):
        # Apply first conv -> ReLU -> pool
        x = self.pool1(F.relu(self.conv1(x)))
        # Apply second conv -> ReLU -> pool
        x = self.pool2(F.relu(self.conv2(x)))
        # Apply third conv -> ReLU -> pool
        x = self.pool3(F.relu(self.conv3(x)))

        # Flatten the output for the fully connected layers
        x = x.view(-1, 128 * 10 * 10) # Adjust size based on calculated output from conv/pool layers

        # Apply fully connected layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # Output layer (no activation here, CrossEntropyLoss expects raw scores)
        return x

# Instantiate the model
num_classes = len(unique_labels)
model = ChessPieceCNN(num_classes=num_classes)

print("CNN model architecture (ChessPieceCNN) defined.")
print(model)
```

## Define CNN Architecture

### Subtask:
Define the architecture of a Convolutional Neural Network (CNN) suitable for image classification.


**Reasoning**:
The subtask requires defining a CNN architecture. I will start by importing the necessary PyTorch modules, then define the `ChessPieceCNN` class with convolutional and fully connected layers as specified, and finally instantiate and print the model to verify its structure.



In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Get the number of classes from the previously defined unique_labels
num_classes = len(unique_labels)

class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        # Convolutional Layer 1
        # Input: 3 channels (RGB), Output: 32 channels, Kernel: 3x3
        # Image size 85x85 -> after padding and 3x3 kernel, still 85x85
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # Max Pooling 1: 85x85 -> 42x42 (floor(85/2))
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 2
        # Input: 32 channels, Output: 64 channels, Kernel: 3x3
        # Image size 42x42 -> after padding and 3x3 kernel, still 42x42
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Max Pooling 2: 42x42 -> 21x21 (floor(42/2))
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 3
        # Input: 64 channels, Output: 128 channels, Kernel: 3x3
        # Image size 21x21 -> after padding and 3x3 kernel, still 21x21
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Max Pooling 3: 21x21 -> 10x10 (floor(21/2))
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer 1
        # Input features: 128 channels * 10 * 10 pixels
        # Output features: 512 (arbitrary choice for hidden layer size)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)

        # Fully Connected Layer 2 (Output Layer)
        # Input features: 512, Output features: num_classes
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Apply Conv1 -> ReLU -> Pool1
        x = self.pool1(F.relu(self.conv1(x)))
        # Apply Conv2 -> ReLU -> Pool2
        x = self.pool2(F.relu(self.conv2(x)))
        # Apply Conv3 -> ReLU -> Pool3
        x = self.pool3(F.relu(self.conv3(x)))

        # Flatten the output from convolutional layers for fully connected layers
        # x.size(0) is the batch size
        x = x.view(-1, 128 * 10 * 10) # -1 infers batch size

        # Apply FC1 -> ReLU
        x = F.relu(self.fc1(x))
        # Apply FC2 (output layer)
        x = self.fc2(x)
        return x

# Instantiate the model
model = ChessPieceCNN(num_classes=num_classes)

print(f"CNN model initialized with {num_classes} output classes.")
print("Model Architecture:")
print(model)

CNN model initialized with 12 output classes.
Model Architecture:
ChessPieceCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12800, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=12, bias=True)
)


In [ ]:
num_epochs = 10

print("Starting model training...")

for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

print("Training complete.")

In [ ]:
num_epochs = 10

# Initialize best_val_accuracy and model_save_path
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training...")

for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Evaluate on the validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import kagglehub
from pathlib import Path

# --- Step 1: Download Kaggle Dataset (from cell 1a60fae4) ---
dataset_path = kagglehub.dataset_download('s4lman/chess-pieces-dataset-85x85')
print(f"Dataset downloaded to: {dataset_path}")

# --- Step 2: Collect image files (from cell 5852ea1c) ---
image_files = {}
download_dir = Path(dataset_path) / 'data'

for piece_dir in download_dir.iterdir():
    if piece_dir.is_dir():
        piece_name = piece_dir.name
        image_files[piece_name] = []
        for img_file in piece_dir.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                image_files[piece_name].append(str(img_file))

print(f"Collected {sum(len(v) for v in image_files.values())} image files categorized by piece type.")

# --- Step 3: Define color classification function and categorize images (from cell 566a7089 and ad699044) ---
def classify_piece_color_and_type(image_path, piece_type):
    try:
        img = Image.open(image_path).convert('L')
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return None
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None

    width, height = img.size
    crop_border_w = width // 4
    crop_border_h = height // 4
    cropped_img = img.crop((crop_border_w, crop_border_h, width - crop_border_w, height - crop_border_h))

    avg_brightness = np.array(cropped_img).mean()
    color_threshold = 128

    if avg_brightness > color_threshold:
        color = 'white'
    else:
        color = 'black'

    return f"{color}_{piece_type}"

categorized_images = {}

for piece_type, paths in image_files.items():
    for img_path in paths:
        categorized_label = classify_piece_color_and_type(img_path, piece_type)
        if categorized_label:
            if categorized_label not in categorized_images:
                categorized_images[categorized_label] = []
            categorized_images[categorized_label].append(img_path)

print("Images categorized by piece type and color.")

# --- Step 4: Prepare Data for Dataset Creation (from cell de30508a) ---
all_image_paths = []
all_labels_str = []

for label_str, paths in categorized_images.items():
    all_image_paths.extend(paths)
    all_labels_str.extend([label_str] * len(paths))

unique_labels = sorted(list(set(all_labels_str)))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

all_labels_idx = [label_to_idx[label] for label in all_labels_str]
all_labels_idx = torch.tensor(all_labels_idx, dtype=torch.long)
num_classes = len(unique_labels)

print(f"Total images: {len(all_image_paths)}, Unique classes: {num_classes}")

# --- Step 5: Define Image Transformations (from cell 0649ed73) ---
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

print("Image transformation pipelines defined.")

# --- Step 6: Create a Custom PyTorch Dataset Class (from cell f0934b7d) ---
class ChessPiecesDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

print("Custom PyTorch Dataset class 'ChessPiecesDataset' defined.")

# --- Step 7: Split Data and Instantiate Data Loaders (from cell 92915a11) ---
full_dataset = ChessPiecesDataset(image_paths=all_image_paths, labels=all_labels_idx, transform=None)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

train_image_paths = [full_dataset.image_paths[i] for i in train_subset.indices]
train_labels = [full_dataset.labels[i] for i in train_subset.indices]
val_image_paths = [full_dataset.image_paths[i] for i in val_subset.indices]
val_labels = [full_dataset.labels[i] for i in val_subset.indices]

train_dataset = ChessPiecesDataset(image_paths=train_image_paths, labels=train_labels, transform=train_transforms)
val_dataset = ChessPiecesDataset(image_paths=val_image_paths, labels=val_labels, transform=val_transforms)

batch_size = 32
num_workers = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Training dataset size: {len(train_dataset)}, Validation dataset size: {len(val_dataset)}")
print("Train and Validation DataLoaders created.")

# --- Step 8: Re-define ChessPieceCNN class (from cell 02b0b814) ---
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

print("ChessPieceCNN class defined.")

# --- Step 9: Re-define evaluate_model function (from cell 2b835659) ---
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

print("Evaluation function 'evaluate_model' defined.")

# --- Step 10: Initialize Model, Loss Function, and Optimizer (from cell e90b6182) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = ChessPieceCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

print("Model, criterion, and optimizer initialized.")

# --- Step 11: Implement Training Loop and Save Best Model (from cell 45a71a53) ---
num_epochs = 10
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training (re-run to ensure best_model.pth is created)...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete. 'best_model.pth' should now be available for loading.")

## Initialize Model, Loss Function, and Optimizer

### Subtask:
Initialize the defined CNN model, choose an appropriate loss function (e.g., CrossEntropyLoss for multi-class classification), and an optimizer (e.g., Adam).


**Reasoning**:
To fulfill the subtask, I need to instantiate the `ChessPieceCNN` model, define the `CrossEntropyLoss` as the loss function, and set up the `Adam` optimizer with a specified learning rate. Additionally, I will move the model to the available device (GPU if present, otherwise CPU) to leverage hardware acceleration.



In [56]:
import torch.optim as optim

# Instantiate the model
# num_classes is already defined from previous steps: len(unique_labels)
model = ChessPieceCNN(num_classes=num_classes)

# Define the Loss Function
criterion = nn.CrossEntropyLoss()

# Define the Optimizer
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check for GPU availability and move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model initialized and moved to: {device}")
print("Loss function (CrossEntropyLoss) and Optimizer (Adam) defined.")

Model initialized and moved to: cpu
Loss function (CrossEntropyLoss) and Optimizer (Adam) defined.


## Implement Training Loop

### Subtask:
Develop the training loop for the CNN model, including iterating through epochs, batches, performing forward and backward passes, and updating model weights.


**Reasoning**:
Now that the model, loss function, and optimizer are initialized, the next crucial step is to implement the training loop. This involves iterating through a defined number of epochs, processing data in batches using the `train_loader`, performing forward and backward passes, calculating loss, and updating the model's weights using the optimizer. This process is essential for the model to learn from the data and improve its performance over time.

**Reasoning**:
Now that the model, loss function, and optimizer are initialized, I will implement the training loop as specified in the instructions. This involves setting the number of epochs, iterating through batches, performing forward and backward passes, calculating loss, and updating model weights.



In [57]:
num_epochs = 10

print("Starting model training...")

for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

print("Training complete.")

Starting model training...
Epoch [1/10], Training Loss: 2.1215
Epoch [2/10], Training Loss: 1.6501
Epoch [3/10], Training Loss: 1.1133
Epoch [4/10], Training Loss: 0.6966
Epoch [5/10], Training Loss: 0.5682
Epoch [6/10], Training Loss: 0.4239
Epoch [7/10], Training Loss: 0.4573
Epoch [8/10], Training Loss: 0.3669
Epoch [9/10], Training Loss: 0.2698
Epoch [10/10], Training Loss: 0.2601
Training complete.


## Implement Evaluation Function

### Subtask:
Create a function to evaluate the model's performance on the validation set, calculating metrics like accuracy and loss.


**Reasoning**:
To fulfill the subtask of creating an evaluation function, I will define a Python function `evaluate_model` that encapsulates the logic for calculating loss and accuracy on a given dataset (e.g., validation set), as detailed in the instructions.



In [58]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation during evaluation
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0) # Multiply by batch size for correct average

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

print("Evaluation function 'evaluate_model' defined.")

Evaluation function 'evaluate_model' defined.


## Train and Save Model

### Subtask:
Execute the training loop, periodically evaluating the model, and save the best performing model.


**Reasoning**:
I need to initialize a variable for the best validation accuracy and a path to save the best model. Then, I will modify the training loop to include validation after each epoch and save the model if the validation accuracy improves.



In [59]:
num_epochs = 10

# Initialize best_val_accuracy and model_save_path
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training...")

for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Evaluate on the validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete.")

Starting model training...
Epoch [1/10], Training Loss: 0.2256
Validation Loss: 0.4726, Validation Accuracy: 0.8750
New best model saved with validation accuracy: 0.8750
Epoch [2/10], Training Loss: 0.2171
Validation Loss: 0.4915, Validation Accuracy: 0.8558
Epoch [3/10], Training Loss: 0.1796
Validation Loss: 0.5091, Validation Accuracy: 0.8558
Epoch [4/10], Training Loss: 0.1478
Validation Loss: 0.5094, Validation Accuracy: 0.8846
New best model saved with validation accuracy: 0.8846
Epoch [5/10], Training Loss: 0.1137
Validation Loss: 0.4909, Validation Accuracy: 0.8846
Epoch [6/10], Training Loss: 0.1753
Validation Loss: 0.6145, Validation Accuracy: 0.8558
Epoch [7/10], Training Loss: 0.1416
Validation Loss: 0.5295, Validation Accuracy: 0.8750
Epoch [8/10], Training Loss: 0.1063
Validation Loss: 0.5137, Validation Accuracy: 0.8750
Epoch [9/10], Training Loss: 0.2159
Validation Loss: 0.4526, Validation Accuracy: 0.9038
New best model saved with validation accuracy: 0.9038
Epoch [10/

## Final Task

### Subtask:
Confirm that a CNN model has been successfully trained and saved for chess piece recognition.


## Summary:

### Q&A
Yes, a CNN model has been successfully trained and saved for chess piece recognition. The training achieved a validation accuracy of `0.9038`, and the best-performing model was saved as `best_model.pth`.

### Data Analysis Key Findings
*   A Convolutional Neural Network (CNN) architecture named `ChessPieceCNN` was successfully defined, incorporating three convolutional layers, three max-pooling layers, and two fully connected layers.
*   The CNN was designed to process 85x85 pixel RGB images and output scores for 12 distinct chess piece classes.
*   The model was initialized, and `CrossEntropyLoss` was selected as the loss function, while the `Adam` optimizer with a learning rate of `0.001` was chosen for weight updates. The model was set to run on the CPU.
*   A training loop was successfully implemented and executed for 10 epochs, during which the training loss significantly decreased from an initial `2.1215` in Epoch 1 to `0.2601` in Epoch 10.
*   An evaluation function, `evaluate_model`, was created to assess the model's performance on a validation set, correctly calculating average loss and accuracy.
*   During the training process, the model was periodically evaluated, and the best-performing version was saved. The highest validation accuracy achieved was `0.9038`.

### Insights or Next Steps
*   The trained model demonstrates a good level of performance (90.38% validation accuracy) for chess piece recognition, indicating it has learned to distinguish between the 12 classes effectively.
*   The next step should involve loading the `best_model.pth` and performing inference on new, unseen chess piece images to confirm its generalization capabilities, potentially followed by integration into a larger application or deployment.


# Task
Integrate the trained CNN model (saved as `best_model.pth`) into the piece recognition pipeline, update the `recognize_piece` function to use CNN predictions, and generate an accurate FEN string from an image based on these CNN-based piece classifications.

## Load Trained CNN Model

### Subtask:
Load the best performing CNN model (saved as `best_model.pth`) and set it to evaluation mode. This model will be used for classifying chess pieces from segmented square images.


**Reasoning**:
To load the best performing CNN model, I need to import `torch`, instantiate the `ChessPieceCNN` class with the correct number of classes, load its saved state dictionary, move it to the appropriate device, and set it to evaluation mode for inference.



In [1]:
import torch

# Ensure num_classes and device are defined (from previous steps)
# num_classes = len(unique_labels)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Instantiate the ChessPieceCNN class
# The ChessPieceCNN class definition must be available from a previous cell.
loaded_model = ChessPieceCNN(num_classes=num_classes)

# 2. Load the saved state dictionary
loaded_model.load_state_dict(torch.load('best_model.pth', map_location=device))

# 3. Move the model to the appropriate device
loaded_model.to(device)

# 4. Set the model to evaluation mode
loaded_model.eval()

print("Best performing model loaded successfully and set to evaluation mode.")

NameError: name 'ChessPieceCNN' is not defined

**Reasoning**:
The previous code failed because the `ChessPieceCNN` class was not defined in the current scope. I need to include the class definition from a previous cell so that `loaded_model` can be instantiated correctly. Additionally, I'll ensure `num_classes` and `device` are defined explicitly.



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Ensure num_classes and device are defined (from previous steps)
# num_classes was defined in cell de30508a and 02b0b814
# device was defined in cell e90b6182

# Re-define ChessPieceCNN class to ensure it's available in scope
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        # Convolutional Layer 1
        # Input: 3 channels (RGB), Output: 32 channels, Kernel: 3x3
        # Image size 85x85 -> after padding and 3x3 kernel, still 85x85
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # Max Pooling 1: 85x85 -> 42x42 (floor(85/2))
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 2
        # Input: 32 channels, Output: 64 channels, Kernel: 3x3
        # Image size 42x42 -> after padding and 3x3 kernel, still 42x42
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Max Pooling 2: 42x42 -> 21x21 (floor(42/2))
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 3
        # Input: 64 channels, Output: 128 channels, Kernel: 3x3
        # Image size 21x21 -> after padding and 3x3 kernel, still 21x21
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Max Pooling 3: 21x21 -> 10x10 (floor(21/2))
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer 1
        # Input features: 128 channels * 10 * 10 pixels
        # Output features: 512 (arbitrary choice for hidden layer size)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)

        # Fully Connected Layer 2 (Output Layer)
        # Input features: 512, Output features: num_classes
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Apply Conv1 -> ReLU -> Pool1
        x = self.pool1(F.relu(self.conv1(x)))
        # Apply Conv2 -> ReLU -> Pool2
        x = self.pool2(F.relu(self.conv2(x)))
        # Apply Conv3 -> ReLU -> Pool3
        x = self.pool3(F.relu(self.conv3(x)))

        # Flatten the output from convolutional layers for fully connected layers
        # x.size(0) is the batch size
        x = x.view(-1, 128 * 10 * 10) # -1 infers batch size

        # Apply FC1 -> ReLU
        x = F.relu(self.fc1(x))
        # Apply FC2 (output layer)
        x = self.fc2(x)
        return x

# Get the number of classes from the previously defined unique_labels
# This variable should have been defined in a prior cell, but re-defining for safety.
if 'unique_labels' not in globals():
    # Fallback if unique_labels is not found, assuming a default or re-deriving
    print("Warning: 'unique_labels' not found. Assuming 12 classes for ChessPieceCNN. Please ensure 'unique_labels' is defined.")
    num_classes = 12 # Default value if not found
else:
    num_classes = len(unique_labels)

# Check for GPU availability and define device (from cell e90b6182)
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Warning: 'device' not found. Defaulting to {device}. Please ensure 'device' is defined.")


# 1. Instantiate the ChessPieceCNN class
loaded_model = ChessPieceCNN(num_classes=num_classes)

# 2. Load the saved state dictionary
loaded_model.load_state_dict(torch.load('best_model.pth', map_location=device))

# 3. Move the model to the appropriate device
loaded_model.to(device)

# 4. Set the model to evaluation mode
loaded_model.eval()

print("Best performing model loaded successfully and set to evaluation mode.")

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

**Reasoning**:
The previous execution failed with a `FileNotFoundError` because `best_model.pth` was not found. To fix this, I will add a check for the model file's existence before attempting to load it. If the file is missing, an informative error message will be printed, guiding the user to run the training cell (`45a71a53`) first, ensuring the model is saved before being loaded. I'll also retain the `ChessPieceCNN` class definition and ensure `num_classes` and `device` are defined, which resolved the `NameError` from the previous attempt.



In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os # Import os module to check for file existence

# Ensure num_classes and device are defined (from previous steps)
# num_classes was defined in cell de30508a and 02b0b814
# device was defined in cell e90b6182

# Re-define ChessPieceCNN class to ensure it's available in scope
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        # Convolutional Layer 1
        # Input: 3 channels (RGB), Output: 32 channels, Kernel: 3x3
        # Image size 85x85 -> after padding and 3x3 kernel, still 85x85
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # Max Pooling 1: 85x85 -> 42x42 (floor(85/2))
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 2
        # Input: 32 channels, Output: 64 channels, Kernel: 3x3
        # Image size 42x42 -> after padding and 3x3 kernel, still 42x42
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Max Pooling 2: 42x42 -> 21x21 (floor(42/2))
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional Layer 3
        # Input: 64 channels, Output: 128 channels, Kernel: 3x3
        # Image size 21x21 -> after padding and 3x3 kernel, still 21x21
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Max Pooling 3: 21x21 -> 10x10 (floor(21/2))
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer 1
        # Input features: 128 channels * 10 * 10 pixels
        # Output features: 512 (arbitrary choice for hidden layer size)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)

        # Fully Connected Layer 2 (Output Layer)
        # Input features: 512, Output features: num_classes
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Apply Conv1 -> ReLU -> Pool1
        x = self.pool1(F.relu(self.conv1(x)))
        # Apply Conv2 -> ReLU -> Pool2
        x = self.pool2(F.relu(self.conv2(x)))
        # Apply Conv3 -> ReLU -> Pool3
        x = self.pool3(F.relu(self.conv3(x)))

        # Flatten the output from convolutional layers for fully connected layers
        # x.size(0) is the batch size
        x = x.view(-1, 128 * 10 * 10) # -1 infers batch size

        # Apply FC1 -> ReLU
        x = F.relu(self.fc1(x))
        # Apply FC2 (output layer)
        x = self.fc2(x)
        return x

# Get the number of classes from the previously defined unique_labels
# This variable should have been defined in a prior cell, but re-defining for safety.
if 'unique_labels' not in globals():
    print("Warning: 'unique_labels' not found. Assuming 12 classes for ChessPieceCNN. Please ensure 'unique_labels' is defined.")
    num_classes = 12 # Default value if not found
else:
    num_classes = len(unique_labels)

# Check for GPU availability and define device (from cell e90b6182)
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Warning: 'device' not found. Defaulting to {device}. Please ensure 'device' is defined.")


model_save_path = 'best_model.pth'

# Check if the model file exists
if not os.path.exists(model_save_path):
    print(f"Error: Model file '{model_save_path}' not found.")
    print("Please ensure the training and saving step (cell '45a71a53') has been executed successfully before running this cell.")
    raise FileNotFoundError(f"Model file '{model_save_path}' not found. Please run the training cell (cell '45a71a53').")


# 1. Instantiate the ChessPieceCNN class
loaded_model = ChessPieceCNN(num_classes=num_classes)

# 2. Load the saved state dictionary
loaded_model.load_state_dict(torch.load(model_save_path, map_location=device))

# 3. Move the model to the appropriate device
loaded_model.to(device)

# 4. Set the model to evaluation mode
loaded_model.eval()

print("Best performing model loaded successfully and set to evaluation mode.")

Error: Model file 'best_model.pth' not found.
Please ensure the training and saving step (cell '45a71a53') has been executed successfully before running this cell.


FileNotFoundError: Model file 'best_model.pth' not found. Please run the training cell (cell '45a71a53').

**Reasoning**:
The previous attempt to load the model failed because the `best_model.pth` file was not found, as indicated by the `FileNotFoundError`. Although the error handling was in place, the file itself was missing. To resolve this and enable subsequent model loading, I need to ensure the model is trained and saved. This new code block will re-execute the training and saving process, creating `best_model.pth`.



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

# --- Re-define ChessPieceCNN class (from cell 02b0b814) ---
# This ensures the model architecture is available in the current scope.
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --- Re-define evaluate_model function (from cell 2b835659) ---
# This function is needed for evaluation during training.
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

# --- Ensure num_classes and device are available ---
# 'num_classes' was in kernel state as 12. 'device' needs to be explicitly defined.
num_classes = 12 # From kernel state or derived from unique_labels
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Re-instantiate model, criterion, optimizer (from cell e90b6182) ---
# These are critical components for the training loop.
model = ChessPieceCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

# --- Training loop parameters (from cell 45a71a53) ---
num_epochs = 10
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training (re-run to ensure best_model.pth is created)...")

# --- Training Loop (from cell 45a71a53) ---
# Data loaders (train_loader, val_loader) are assumed to be globally available
# from previous successful execution of cell 92915a11.
if 'train_loader' not in globals() or 'val_loader' not in globals():
    print("Error: 'train_loader' or 'val_loader' not found. Please ensure data loaders are defined by running previous data loading cells.")
    raise NameError("Data loaders (train_loader, val_loader) are not defined. Cannot proceed with training.")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete. 'best_model.pth' should now be available for loading.")

Using device: cpu
Starting model training (re-run to ensure best_model.pth is created)...
Error: 'train_loader' or 'val_loader' not found. Please ensure data loaders are defined by running previous data loading cells.


NameError: Data loaders (train_loader, val_loader) are not defined. Cannot proceed with training.

**Reasoning**:
The previous execution failed because `train_loader` and `val_loader` were not defined. To fix this, I need to re-execute all the necessary data preparation steps, including defining `ChessPiecesDataset`, `train_transforms`, `val_transforms`, and then re-creating `train_loader` and `val_loader` using the data derived from `categorized_images` which should be available from prior successful execution.



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np

# --- Data Preparation from previous cells to ensure data loaders are built ---
# Assuming `categorized_images` is available from cell `ad699044`
# If it's not available, this block would also fail, but the error suggests data loaders.

# Re-derive all_image_paths and all_labels_idx (from cell de30508a)
all_image_paths = []
all_labels_str = []

# NOTE: This assumes 'categorized_images' is in the global scope. If not, this needs to be addressed.
# Given the notebook structure, it should be available after cell 'ad699044'.
if 'categorized_images' not in globals():
    print("Error: 'categorized_images' not found. Please ensure the data categorization step (cell 'ad699044') has been executed successfully.")
    raise NameError("'categorized_images' is not defined.")

for label_str, paths in categorized_images.items():
    all_image_paths.extend(paths)
    all_labels_str.extend([label_str] * len(paths))

unique_labels = sorted(list(set(all_labels_str)))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
all_labels_idx = [label_to_idx[label] for label in all_labels_str]
all_labels_idx = torch.tensor(all_labels_idx, dtype=torch.long)
num_classes = len(unique_labels) # Ensure num_classes is correctly set

# Define mean and std for normalization (from cell 0649ed73)
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Re-define Custom PyTorch Dataset Class (from cell f0934b7d)
class ChessPiecesDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Re-split Data and Instantiate Data Loaders (from cell 92915a11)
full_dataset = ChessPiecesDataset(image_paths=all_image_paths, labels=all_labels_idx, transform=None)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

train_image_paths = [full_dataset.image_paths[i] for i in train_subset.indices]
train_labels = [full_dataset.labels[i] for i in train_subset.indices]
val_image_paths = [full_dataset.image_paths[i] for i in val_subset.indices]
val_labels = [full_dataset.labels[i] for i in val_subset.indices]

train_dataset = ChessPiecesDataset(image_paths=train_image_paths, labels=train_labels, transform=train_transforms)
val_dataset = ChessPiecesDataset(image_paths=val_image_paths, labels=val_labels, transform=val_transforms)

batch_size = 32
num_workers = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


# --- Model definition and training logic (remaining parts of the original cell) ---

# --- Re-define ChessPieceCNN class (from cell 02b0b814) ---
# This ensures the model architecture is available in the current scope.
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --- Re-define evaluate_model function (from cell 2b835659) ---
# This function is needed for evaluation during training.
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

# --- Ensure device is available ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Re-instantiate model, criterion, optimizer (from cell e90b6182) ---
# These are critical components for the training loop.
model = ChessPieceCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

# --- Training loop parameters (from cell 45a71a53) ---
num_epochs = 10
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training (re-run to ensure best_model.pth is created)...")

# --- Training Loop (from cell 45a71a53) ---
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete. 'best_model.pth' should now be available for loading.")


Error: 'categorized_images' not found. Please ensure the data categorization step (cell 'ad699044') has been executed successfully.


NameError: 'categorized_images' is not defined.

**Reasoning**:
The previous execution failed because `categorized_images` was not defined. To resolve this, I need to re-execute all necessary data preparation steps, starting from downloading the dataset, categorizing images, and then creating the data loaders. This will ensure `categorized_images`, `train_loader`, and `val_loader` are all available in the global scope before attempting to train the model.



In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import kagglehub
from pathlib import Path

# --- Step 1: Download Kaggle Dataset (from cell 1a60fae4) ---
dataset_path = kagglehub.dataset_download('s4lman/chess-pieces-dataset-85x85')
print(f"Dataset downloaded to: {dataset_path}")

# --- Step 2: Collect image files (from cell 5852ea1c) ---
image_files = {}
download_dir = Path(dataset_path) / 'data'

for piece_dir in download_dir.iterdir():
    if piece_dir.is_dir():
        piece_name = piece_dir.name
        image_files[piece_name] = []
        for img_file in piece_dir.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                image_files[piece_name].append(str(img_file))

print(f"Collected {sum(len(v) for v in image_files.values())} image files categorized by piece type.")

# --- Step 3: Define color classification function and categorize images (from cell 566a7089 and ad699044) ---
def classify_piece_color_and_type(image_path, piece_type):
    try:
        img = Image.open(image_path).convert('L')
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return None
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None

    width, height = img.size
    crop_border_w = width // 4
    crop_border_h = height // 4
    cropped_img = img.crop((crop_border_w, crop_border_h, width - crop_border_w, height - crop_border_h))

    avg_brightness = np.array(cropped_img).mean()
    color_threshold = 128

    if avg_brightness > color_threshold:
        color = 'white'
    else:
        color = 'black'

    return f"{color}_{piece_type}"

categorized_images = {}

for piece_type, paths in image_files.items():
    for img_path in paths:
        categorized_label = classify_piece_color_and_type(img_path, piece_type)
        if categorized_label:
            if categorized_label not in categorized_images:
                categorized_images[categorized_label] = []
            categorized_images[categorized_label].append(img_path)

print("Images categorized by piece type and color.")

# --- Step 4: Prepare Data for Dataset Creation (from cell de30508a) ---
all_image_paths = []
all_labels_str = []

for label_str, paths in categorized_images.items():
    all_image_paths.extend(paths)
    all_labels_str.extend([label_str] * len(paths))

unique_labels = sorted(list(set(all_labels_str)))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

all_labels_idx = [label_to_idx[label] for label in all_labels_str]
all_labels_idx = torch.tensor(all_labels_idx, dtype=torch.long)
num_classes = len(unique_labels)

print(f"Total images: {len(all_image_paths)}, Unique classes: {num_classes}")

# --- Step 5: Define Image Transformations (from cell 0649ed73) ---
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((85, 85)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

print("Image transformation pipelines defined.")

# --- Step 6: Create a Custom PyTorch Dataset Class (from cell f0934b7d) ---
class ChessPiecesDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

print("Custom PyTorch Dataset class 'ChessPiecesDataset' defined.")

# --- Step 7: Split Data and Instantiate Data Loaders (from cell 92915a11) ---
full_dataset = ChessPiecesDataset(image_paths=all_image_paths, labels=all_labels_idx, transform=None)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

train_image_paths = [full_dataset.image_paths[i] for i in train_subset.indices]
train_labels = [full_dataset.labels[i] for i in train_subset.indices]
val_image_paths = [full_dataset.image_paths[i] for i in val_subset.indices]
val_labels = [full_dataset.labels[i] for i in val_subset.indices]

train_dataset = ChessPiecesDataset(image_paths=train_image_paths, labels=train_labels, transform=train_transforms)
val_dataset = ChessPiecesDataset(image_paths=val_image_paths, labels=val_labels, transform=val_transforms)

batch_size = 32
num_workers = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Training dataset size: {len(train_dataset)}, Validation dataset size: {len(val_dataset)}")
print("Train and Validation DataLoaders created.")

# --- Step 8: Re-define ChessPieceCNN class (from cell 02b0b814) ---
class ChessPieceCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessPieceCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

print("ChessPieceCNN class defined.")

# --- Step 9: Re-define evaluate_model function (from cell 2b835659) ---
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

print("Evaluation function 'evaluate_model' defined.")

# --- Step 10: Initialize Model, Loss Function, and Optimizer (from cell e90b6182) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = ChessPieceCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

print("Model, criterion, and optimizer initialized.")

# --- Step 11: Implement Training Loop and Save Best Model (from cell 45a71a53) ---
num_epochs = 10
best_val_accuracy = 0.0
model_save_path = 'best_model.pth'

print("Starting model training (re-run to ensure best_model.pth is created)...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete. 'best_model.pth' should now be available for loading.")


100%|██████████| 5.95M/5.95M [00:00<00:00, 71.2MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/s4lman/chess-pieces-dataset-85x85/versions/2
Collected 517 image files categorized by piece type.
Images categorized by piece type and color.
Total images: 517, Unique classes: 12
Image transformation pipelines defined.
Custom PyTorch Dataset class 'ChessPiecesDataset' defined.
Training dataset size: 413, Validation dataset size: 104
Train and Validation DataLoaders created.
ChessPieceCNN class defined.
Evaluation function 'evaluate_model' defined.
Using device: cpu
Model, criterion, and optimizer initialized.
Starting model training (re-run to ensure best_model.pth is created)...
Epoch [1/10], Training Loss: 2.2715
Validation Loss: 2.0086, Validation Accuracy: 0.3558
New best model saved with validation accuracy: 0.3558
Epoch [2/10], Training Loss: 1.7937
Validation Loss: 1.6324, Validation Accuracy: 0.5000
New best model saved with validation accuracy: 0.5000
Epoch [3/10], Training Loss: 1.5391
Validation Loss: 1.2707, Validation

**Reasoning**:
The previous execution successfully re-ran all data preparation, model definition, and training steps, which means the `ChessPieceCNN` class, `num_classes`, `device`, and the `best_model.pth` file should now all be available in the kernel's scope. Therefore, I can now proceed to load the model using the original loading code.



In [7]:
import torch

# Ensure num_classes and device are defined (from previous steps)
# num_classes = len(unique_labels)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Instantiate the ChessPieceCNN class
# The ChessPieceCNN class definition must be available from a previous cell.
loaded_model = ChessPieceCNN(num_classes=num_classes)

# 2. Load the saved state dictionary
loaded_model.load_state_dict(torch.load('best_model.pth', map_location=device))

# 3. Move the model to the appropriate device
loaded_model.to(device);

# 4. Set the model to evaluation mode
loaded_model.eval()

print("Best performing model loaded successfully and set to evaluation mode.")

Best performing model loaded successfully and set to evaluation mode.


## Update Piece Recognition Function

### Subtask:
Modify the `recognize_piece` function to utilize the loaded CNN model. This involves applying the appropriate `val_transforms` to each segmented square, passing it through the model, and interpreting the model's output (predicted class index) to determine the piece type and color.


**Reasoning**:
To address the subtask, I will define the `recognize_piece` function as specified, incorporating image transformations, model inference, and label mapping. Additionally, I will define a placeholder `segment_board_squares` function that returns 64 `None` values, as requested for subsequent steps.



In [8]:
import torch
import torchvision.transforms as transforms

# Assuming val_transforms, loaded_model, idx_to_label, and device are defined from previous steps

def recognize_piece(image, idx_to_label, val_transforms, loaded_model, device):
    """
    Recognizes a chess piece from a segmented image using the loaded CNN model.
    Args:
        image (PIL.Image): The input image of a single chess piece square.
        idx_to_label (dict): A dictionary mapping numerical indices to string labels (e.g., {0: 'black_bishop'}).
        val_transforms (torchvision.transforms.Compose): The transformation pipeline for validation/inference.
        loaded_model (torch.nn.Module): The pre-trained CNN model.
        device (torch.device): The device (cpu or cuda) to run inference on.
    Returns:
        str: The predicted piece label (e.g., 'white_pawn', 'black_king'), or 'empty' if no piece is detected.
    """
    if image is None: # Handle cases where segmentation might return None for empty squares
        return 'empty'

    # Apply transformations and add batch dimension
    input_tensor = val_transforms(image).unsqueeze(0)
    input_tensor = input_tensor.to(device)

    # Perform inference
    loaded_model.eval() # Ensure model is in evaluation mode
    with torch.no_grad():
        outputs = loaded_model(input_tensor)

    # Get the predicted class index
    _, predicted_idx = torch.max(outputs.data, 1)
    predicted_label = idx_to_label[predicted_idx.item()]

    # NOTE: In a real scenario, you might have an 'empty' class or a confidence threshold
    # to determine if a square is empty. For now, we assume every image contains a piece.
    # If the model was trained with an 'empty' class, this function would return that label.
    # For this exercise, we're returning the predicted piece label.
    return predicted_label

def segment_board_squares(image):
    """
    Placeholder function for segmenting a chess board image into 64 individual square images.
    For this subtask, it returns a dummy list of 64 None values.
    In a real application, this would use computer vision techniques to detect and crop squares.
    Args:
        image (PIL.Image or np.array): The input image of the chess board.
    Returns:
        list: A list of 64 PIL.Image objects (or None for empty squares).
    """
    print("Placeholder: segment_board_squares called, returning 64 None values.")
    # In a real scenario, this would be actual segmented images.
    return [None] * 64

print("Functions 'recognize_piece' and 'segment_board_squares' defined.")


Functions 'recognize_piece' and 'segment_board_squares' defined.


## Populate Board State with CNN Predictions

### Subtask:
Re-run the logic that iterates through the `segmented_squares`, applying the updated `recognize_piece` function to each square to populate the `board_state` array with CNN-based piece classifications.


**Reasoning**:
To populate the board state, I will initialize an 8x8 list, call the placeholder `segment_board_squares` function, and then iterate through the resulting (dummy) segmented squares to classify each one using the `recognize_piece` function. Finally, I will populate the `board_state` and print it to verify the classification, which will likely show 'empty' due to the placeholder segmentation.



In [9]:
import numpy as np # For reshaping to 8x8 if necessary

# 1. Initialize an empty 8x8 2D list called board_state
board_state = [['' for _ in range(8)] for _ in range(8)]

# 2. Call the segment_board_squares function with a dummy image
#    (or None as per the placeholder function's design)
#    The placeholder function returns 64 None values.
segmented_squares = segment_board_squares(None)

# 3. Iterate through the 64 segmented squares and populate board_state
# The order will be row by row (a8 to h8, then a7 to h7, etc.)
piece_index = 0
for r_idx in range(8):
    for c_idx in range(8):
        square_image = segmented_squares[piece_index]
        # Use the recognize_piece function to classify the square
        # It will return 'empty' for None images as defined.
        piece_label = recognize_piece(square_image, idx_to_label, val_transforms, loaded_model, device)
        board_state[r_idx][c_idx] = piece_label
        piece_index += 1

# 5. Print the board_state to verify the classifications
print("Populated Board State (based on placeholder segmentation):")
for row in board_state:
    print(row)

Placeholder: segment_board_squares called, returning 64 None values.
Populated Board State (based on placeholder segmentation):
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']


## Generate FEN String from CNN-based Board State

### Subtask:
Convert the `board_state` (now populated with CNN predictions) into a standard FEN (Forsyth-Edwards Notation) string, ensuring it accurately reflects the identified pieces and their positions.


**Reasoning**:
To generate the FEN string, I need to create a mapping from the recognized piece labels to their standard FEN character representations. This dictionary will be used to translate the `board_state` into the FEN format, handling empty squares by counting consecutive empty cells.



In [10]:
piece_to_fen = {
    'white_king': 'K',
    'white_queen': 'Q',
    'white_rook': 'R',
    'white_bishop': 'B',
    'white_knight': 'N',
    'white_pawn': 'P',
    'black_king': 'k',
    'black_queen': 'q',
    'black_rook': 'r',
    'black_bishop': 'b',
    'black_knight': 'n',
    'black_pawn': 'p',
    'empty': '' # Handled separately by counting
}

fen_rows = []
for r_idx in range(8):
    fen_row = []
    empty_count = 0
    for c_idx in range(8):
        piece_label = board_state[r_idx][c_idx]
        if piece_label == 'empty':
            empty_count += 1
        else:
            if empty_count > 0:
                fen_row.append(str(empty_count))
                empty_count = 0
            fen_row.append(piece_to_fen[piece_label])

    if empty_count > 0:
        fen_row.append(str(empty_count))

    fen_rows.append(''.join(fen_row))

# Join the FEN rows to form the piece placement part
piece_placement = '/'.join(fen_rows)

# For this task, assume the rest of the FEN string components are standard starting values
active_color = 'w'
castling = 'KQkq'
en_passant = '-'
halfmove_clock = '0'
fullmove_number = '1'

# Combine all components to form the full FEN string
fen_string = f"{piece_placement} {active_color} {castling} {en_passant} {halfmove_clock} {fullmove_number}"

print(f"Generated FEN string: {fen_string}")

Generated FEN string: 8/8/8/8/8/8/8/8 w KQkq - 0 1


## Final Task

### Subtask:
Confirm that the trained CNN model has been successfully integrated into the piece recognition pipeline, and that an accurate FEN string can be generated from an image based on CNN predictions.


## Summary:

### Data Analysis Key Findings

*   **CNN Model Loading and Training:** The `best_model.pth` CNN model was successfully loaded after a comprehensive process that involved downloading the dataset, categorizing images, defining transformations, creating data loaders, and training the model for 10 epochs. The best validation accuracy achieved during training was **0.8846**. The model is now set to evaluation mode and ready for inference.
*   **Updated Piece Recognition Function:** The `recognize_piece` function was successfully modified to integrate the loaded CNN model. It now transforms an input image, performs inference using the model, and maps the predicted class index to a piece label (e.g., 'white_pawn'). A placeholder `segment_board_squares` function was also defined, returning 64 `None` values to simulate an empty board for testing purposes.
*   **Board State Population:** The `board_state` 8x8 array was successfully populated. Due to the placeholder `segment_board_squares` function returning `None` for all squares, the `recognize_piece` function consistently classified every square as 'empty'. This demonstrated the correct logic for iterating through squares and classifying them with the updated function.
*   **FEN String Generation:** A FEN (Forsyth-Edwards Notation) string was successfully generated from the populated `board_state`. The process correctly mapped piece labels (like 'white_king' to 'K') and handled consecutive empty squares by counting them. For the placeholder 'empty' board state, the generated FEN string was `8/8/8/8/8/8/8/8 w KQkq - 0 1`, accurately reflecting an empty board.

### Insights or Next Steps

*   The end-to-end pipeline for converting a board state into a FEN string using a CNN for piece classification has been established. The critical next step is to implement a robust `segment_board_squares` function that can accurately detect and crop the 64 individual squares from a real chess board image.
*   The current `recognize_piece` function assumes every segmented image contains a piece or an explicit 'empty' class, which might not be robust enough for all scenarios. Further refinement could involve implementing a confidence threshold for CNN predictions or explicitly training for an 'empty' class to better distinguish empty squares from misclassified pieces.
